# Урок 6. Многоклассовая классификация.

Посмотрим на примере алгоритма логистической регрессии и метода опорных векторов, как работать с различными методами многоклассовой классификации.

### 1.
Вспомните датасет Wine. Загрузите его, разделите на тренировочную и тестовую выборки (random_state=17), используя только [9, 11, 12] признаки.

In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split

In [20]:
### YOUR CODE HERE ###
# НУМЕРАЦИЯ ИМЕЕТСЯ В ВИДУ С НУЛЯ
import numpy as np

ds = load_wine()
x1 = ds.data[:,9:10]
x2 = ds.data[:,11:12]
x3 = ds.data[:,12:]

X = np.hstack([x1,x2,x3])
Y = ds.target
print(ds.data[:3])
print(X[:3])

Xt, Xv, Yt, Yv = train_test_split(X, Y, random_state=17)
len(Xt)

[[1.423e+01 1.710e+00 2.430e+00 1.560e+01 1.270e+02 2.800e+00 3.060e+00
  2.800e-01 2.290e+00 5.640e+00 1.040e+00 3.920e+00 1.065e+03]
 [1.320e+01 1.780e+00 2.140e+00 1.120e+01 1.000e+02 2.650e+00 2.760e+00
  2.600e-01 1.280e+00 4.380e+00 1.050e+00 3.400e+00 1.050e+03]
 [1.316e+01 2.360e+00 2.670e+00 1.860e+01 1.010e+02 2.800e+00 3.240e+00
  3.000e-01 2.810e+00 5.680e+00 1.030e+00 3.170e+00 1.185e+03]]
[[   5.64    3.92 1065.  ]
 [   4.38    3.4  1050.  ]
 [   5.68    3.17 1185.  ]]


133

**Задайте тип кросс-валидации с помощью StratifiedKFold: 5-кратная, random_state=17.**

In [10]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

In [16]:
skf = StratifiedKFold(n_splits=5, random_state=17)



/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


### 2.
Обучите логистическую регрессию (LogisticRegression) с параметром C по умолчанию и random_state=17. Укажите гиперпараметр multi_class='ovr' - по умолчанию многие классификаторы используют именно его. С помощью cross_val_score сделайте кросс-валидацию (используйте объект skf) и выведите среднюю долю правильных ответов на ней (используйте функцию mean). Отдельно выведите долю правильных ответов на тестовой выборке.

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [24]:
### YOUR CODE HERE ###

lr = LogisticRegression(multi_class='ovr', random_state=17)
lr.fit(Xt, Yt)

print("CVS for train:", cross_val_score(lr, X=Xt, y=Yt, cv=skf).mean())
print("CVS for valid:", cross_val_score(lr, X=Xv, y=Yv, cv=skf).mean())


CVS for train: 0.8945868945868944
CVS for valid: 0.9555555555555555


### 3.
Обучите метод опорных векторов (SVC) с random_state=17 и остальными параметрами по умолчанию. Этот метод при мультиклассовой классификации также использует метод "ovr". Сделайте кросс-валидацию (используйте skf) и, как и в предыдущем пункте, выведите среднюю долю правильных ответов на ней. Отдельно выведите долю правильных ответов на тестовой выборке.

In [23]:
from sklearn.svm import SVC

In [25]:
### YOUR CODE HERE ###

svc = SVC(random_state=17)
svc.fit(Xt, Yt)

print("CVS for train:", cross_val_score(svc, X=Xt, y=Yt, cv=skf).mean())
print("CVS for valid:", cross_val_score(svc, X=Xv, y=Yv, cv=skf).mean())

CVS for train: 0.6920227920227922
CVS for valid: 0.7777777777777777


Как видно из полученной метрики, на тестовой выборке метод с гиперпараметрами по умолчанию работает явно намного хуже логистической регрессии. В целом, SVM достаточно плохо масштабируется на размер обучающего набора данных (как видно, даже с тремя признаками он работает не очень хорошо), но благодаря возможности выбора различных ядер (функций близости, которые помогают разделять данные) и другим гиперпараметрам SVM можно достаточно точно настроить под определенный вид данных. Подробнее на этом останавливаться в контексте данного урока не будем.

### 4.
Для предсказаний обеих моделей постройте матрицу ошибок (confusion matrix) и напишите, какие классы каждая из моделей путает больше всего между собой.

In [26]:
from sklearn.metrics import classification_report, confusion_matrix

In [29]:
### YOUR CODE HERE ###

print(confusion_matrix(Yv, lr.predict(Xv)), "\n\n", confusion_matrix(Yv, svc.predict(Xv)))

#Логистическа регрессия путает класс 2 и 3 (если с единицы), SVC точно так же

[[ 9  0  0]
 [ 0 19  0]
 [ 0  4 13]] 

 [[ 9  0  0]
 [ 0 19  0]
 [ 2 15  0]]


### 5.
Для каждой модели выведите classification report.

In [30]:
### YOUR CODE HERE ###

print(classification_report(Yv, lr.predict(Xv)), "\n\n\n", classification_report(Yv, svc.predict(Xv)))




              precision    recall  f1-score   support

           0       1.00      1.00      1.00         9
           1       0.83      1.00      0.90        19
           2       1.00      0.76      0.87        17

    accuracy                           0.91        45
   macro avg       0.94      0.92      0.92        45
weighted avg       0.93      0.91      0.91        45
 


               precision    recall  f1-score   support

           0       0.82      1.00      0.90         9
           1       0.56      1.00      0.72        19
           2       0.00      0.00      0.00        17

    accuracy                           0.62        45
   macro avg       0.46      0.67      0.54        45
weighted avg       0.40      0.62      0.48        45



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
